In [ ]:
# необходимые библиотеки

import requests
import pandas as pd

In [ ]:

# функция для преобразования словаря в data frame 

def request_world_bank_data(countries, indicators, start_date, end_date, verify = None):
    base_url = "https://api.worldbank.org/v2/country/"
    format = "json"
    data = {}

    for country in countries:
        for indicator in indicators:
            url = f"{base_url}{country}/indicator/{indicator}?format={format}&date={start_date}:{end_date}"
            response = requests.get(url, verify=verify)

            if response.status_code == 200:
                json_data = response.json()

                # Достаем данные из JSON 
                country_name = json_data[1][0]['country']['value']
                indicator_name = json_data[1][0]['indicator']['value']
                values = [(entry['date'], entry['value']) for entry in json_data[1] if entry['value'] is not None]

                # Записываем данные в словарь
                if country_name not in data:
                    data[country_name] = {}

                data[country_name][indicator_name] = values

    return data


In [ ]:
# функция для преобразования словаря в data frame 

def create_df(data):
    df = pd.DataFrame()
    for country, indicators in data.items():
        for indicator, values in indicators.items():
            temp_df = pd.DataFrame(values, columns=['Year', indicator])
            temp_df.set_index('Year', inplace=True)
            temp_df.columns = pd.MultiIndex.from_product([[country], [indicator]])
            df = pd.concat([df, temp_df], axis=1)

    df.index.name = 'Year'
    df.sort_index(ascending=True, inplace=True)
    return df

In [ ]:
# задаем параметры функции 

countries = ["USA", "GBR"]
indicators = ["SP.POP.TOTL", "NY.GDP.MKTP.CD"]
start_date = "2015"
end_date = "2020"

data = request_world_bank_data(countries, indicators, start_date, end_date)

df = create_df(data)
df